### CHanges:
1. Included functionality to collect positive, negative, helpful and recent reviews

In [37]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import re

In [38]:
flipkart_base_url = 'http://www.flipkart.com'
user_input = input("Enter the smartphone name: ")
search_string = 'site:www.flipkart.com '+ user_input + ' flipkart'
r = requests.get('https://google.com/search?q='+search_string)
r

<Response [200]>

In [39]:
#Filter the links that directs to flipkart website
def filter_link(s):
    pattern = re.compile(r'(https://www.flipkart.com)(.*)&sa')
    matches = pattern.finditer(s)
    if matches:
        for i in matches:
            target_link = f'{i.group(1)}{i.group(2)}'
            return target_link
    return 'invalid'

In [40]:
soup = BeautifulSoup(r.content, features="html.parser")
link_tags = soup.select('a')
all_links = []
valid_links = []
for i in link_tags:
    all_links.append(i.get('href'))
for link in all_links:
    response = filter_link(link)
    if  response != 'invalid':
        valid_links.append(response)
valid_links[:min(5, len(valid_links))]

['https://www.flipkart.com/poco-m3-cool-blue-64-gb/p/itmc8ec867cb0472',
 'https://www.flipkart.com/poco-m3-yellow-128-gb/p/itm8d6c2240709b5',
 'https://www.flipkart.com/poco-m3-cool-blue-64-gb/p/itm3bcdc32409005',
 'https://www.flipkart.com/poco-m3-power-black-64-gb/p/itmb49cc10841be2',
 'https://www.flipkart.com/poco-m3-yellow-64-gb/p/itmb64ed93540462']

In [41]:
product_page_absolute_url = valid_links[:1]
product_page_absolute_url

['https://www.flipkart.com/poco-m3-cool-blue-64-gb/p/itmc8ec867cb0472']

In [42]:
product_response = requests.get(str(product_page_absolute_url[0]))

In [43]:
product_soup = BeautifulSoup(product_response.content)


### Capture the negative reviews

In [44]:
review_page_relative_url = product_soup.find('a', string=re.compile(r'All.*reviews')).get('href')
review_page_absolute_url = flipkart_base_url + review_page_relative_url
review_page_absolute_url

'http://www.flipkart.com/poco-m3-cool-blue-64-gb/product-reviews/itmc8ec867cb0472?pid=MOBFZTCUDDCTDN3G&lid=LSTMOBFZTCUDDCTDN3GPBGRXL&marketplace=FLIPKART'

In [45]:
# Generate the url of positive, negative, helpful, recent and negative review page url. 
# It ensures varities in the collected data
negative_page_url = review_page_absolute_url.replace('&marketplace=FLIPKART', '&aid=overall&certifiedBuyer=false&sortOrder=NEGATIVE_FIRST')
positive_page_url = review_page_absolute_url.replace('&marketplace=FLIPKART', '&aid=overall&certifiedBuyer=false&sortOrder=POSITIVE_FIRST')
helpful_page_url = review_page_absolute_url.replace('&marketplace=FLIPKART', '&aid=overall&certifiedBuyer=false&sortOrder=MOST_HELPFUL')
recent_page_url = review_page_absolute_url.replace('&marketplace=FLIPKART', '&aid=overall&certifiedBuyer=false&sortOrder=MOST_RECENT')

In [46]:
# Send seperate get requests to them individually
positive_page_response = requests.get(positive_page_url)
negative_page_response = requests.get(negative_page_url)
helpful_page_response = requests.get(helpful_page_url)
recent_page_response = requests.get(recent_page_url)

In [47]:
# Send get request to the first review page
# review_page_response = requests.get(review_page_absolute_url)

In [48]:
#review_page_soup = BeautifulSoup(review_page_response.content)

positive_page_soup = BeautifulSoup(positive_page_response.content)
negative_page_soup = BeautifulSoup(negative_page_response.content)
helpful_page_soup = BeautifulSoup(helpful_page_response.content)
recent_page_soup = BeautifulSoup(recent_page_response.content)

In [49]:
# get links of  5 pages out of 10 available  pages that are available in one page, 
# for the remaining page links we have to got to the next page by clicking on Next button
def get_all_review_page_links(first_page_url, soup):
    all_links_tags = soup.findAll('a', {'class':'ge-49M'})
    num_pages_to_collect = 5
    review_page_links = []
    for link in all_links_tags[:num_pages_to_collect]:
        relative_url = link.get('href')
        absolute_url = flipkart_base_url + relative_url     
        review_page_links.append(absolute_url)
    return review_page_links
#get_all_review_page_links(negative_page_url, negative_page_soup)

In [50]:
review_titles = []
review_descriptions = []
review_ratings = []

In [51]:
reviews_df = pd.DataFrame(columns = ['review_title', 'review_description', 'rating'])
reviews_df.head()

,review_title,review_description,rating


In [52]:
#get the review titles for the page
def get_review_titles(soup):
    titles_tags = soup.findAll('p', {'class':'_2-N8zT'})
    for t in titles_tags:
        review_titles.append(t.text)
    return review_titles
#get_review_titles(review_page_soup)

In [53]:
#get the review descriptions for the page
def get_review_descriptions(soup):
    review_descriptions_tags = soup.findAll('div', {'class':'t-ZTKy'})
    for t in review_descriptions_tags:
        review_descriptions.append(t.get_text().replace('READ MORE', ''))
    return review_descriptions
#get_review_descriptions(review_page_soup)

In [54]:
#get the ratings for the page
def get_review_ratings(soup):
    
    review_ratings_tags = soup.findAll('div', {'class':['_3LWZlK _1rdVr6 _1BLPMq', '_3LWZlK _1BLPMq', '_3LWZlK _32lA32 _1BLPMq']})
    for t in review_ratings_tags:
        review_ratings.append(int(t.text.strip()))
    return review_ratings      
#get_review_ratings(review_page_soup)    

In [55]:
#Fetching only first 5 pages, roughly 50 reviews
def collect_data(url, soup):
    review_page_absolute_url = url
    for link in get_all_review_page_links(review_page_absolute_url, soup):
        review_page_response = requests.get(link)
        review_page_soup = BeautifulSoup(review_page_response.content)
        review_titles = get_review_titles(review_page_soup)
        review_descriptions = get_review_descriptions(review_page_soup)
        review_ratings = get_review_ratings(review_page_soup)
#collect_data(negative_page_url, negative_page_soup)

In [56]:
len(review_ratings), len(review_descriptions), len(review_titles)

(0, 0, 0)

In [57]:
#Collect the positive reviews
collect_data(positive_page_url, positive_page_soup)
temp_df = pd.DataFrame({'review_title':review_titles, 'review_description':review_descriptions, 'rating':review_ratings})
reviews_df = pd.concat([reviews_df, temp_df], axis=0)
time.sleep(3)

#Collect the negative reviews
collect_data(negative_page_url, negative_page_soup)
temp_df = pd.DataFrame({'review_title':review_titles, 'review_description':review_descriptions, 'rating':review_ratings})
reviews_df = pd.concat([reviews_df, temp_df], axis=0)
time.sleep(4)

#Collect the helpful reviews
collect_data(helpful_page_url, helpful_page_soup)
temp_df = pd.DataFrame({'review_title':review_titles, 'review_description':review_descriptions, 'rating':review_ratings})
reviews_df = pd.concat([reviews_df, temp_df], axis=0)
time.sleep(3)

#Collect the recent reviews
collect_data(recent_page_url, recent_page_soup)
temp_df = pd.DataFrame({'review_title':review_titles, 'review_description':review_descriptions, 'rating':review_ratings})
reviews_df = pd.concat([reviews_df, temp_df], axis=0)

In [58]:
reviews_df.shape

(500, 3)

In [59]:
reviews_df.to_csv(f'Reviews/flipkart_reviews_{user_input}.csv', index=False)

In [60]:
test = pd.read_csv(f'Reviews/flipkart_reviews_{user_input}.csv')
test.head()

,review_title,review_description,rating
0,Wonderful,Good phone battery🔋 And camera This price poin...,5
1,Terrific,"One word review "" Value for Money""It has the p...",5
2,Simply awesome,Mobile is good... I bought the yellow one... F...,5
3,Mind-blowing purchase,Thank You Poco for manufacturing this Phone......,5
4,Awesome,I am writing this review after using this mobi...,5


In [61]:
test.shape

(500, 3)